# DirAPI Example 2: Template SQL

In ./Example1.ipynb , we created API to load json and text files.

In this notebook, let us create an API for SQL templates.

In [1]:
# If you have not installed dirapi yet,
# !pip install -e ../../

## Libraries

In [2]:
from datetime import date
import jinja2
import os
import sqlite3 as sql
from typing import Any, Dict, Iterable, List, Tuple

from dirapi import create_api, help_tree

%reload_ext autoreload
%autoreload 2

In [3]:
# const.
DB = "./example.db"
ROOT_DIREC = "../sqls/"

## Tools

In [4]:
def load_txt(path: str) -> str:
    """Load text data
    """
    with open(path) as f:
        return f.read()

In [5]:
def execute_sql(path:str, **kwargs):
    """Execute python script
    """
    with open(path) as f:
        exec(f.read(), globals())

## Preparation

First, let us create a simple database.

This database has three tables: Customer, Transaction1 and Transaction2.

In [6]:
if os.path.exists(DB):
    os.remove(DB)
conn = sql.connect(DB)
cur = conn.cursor()

In [7]:
# create customer table
cur.execute("CREATE TABLE Customer (Customer text, JoinDate date, OutDate date, Age real)")
cur.execute(f'INSERT INTO Customer VALUES ("Alice", "2022-01-15", NULL, 25)')
cur.execute(f'INSERT INTO Customer VALUES ("Bob", "2022-01-20", NULL, 21)')
cur.execute(f'INSERT INTO Customer VALUES ("Charlie", "2022-01-21", NULL, 17)')
conn.commit()

In [8]:
# create transaction1 table
cur.execute("CREATE TABLE Transaction1 (Date date, Customer text, Goods text, Price real)")
cur.execute(f'INSERT INTO Transaction1 VALUES ("2022-01-15", "Alice", "Cup", 100)')
cur.execute(f'INSERT INTO Transaction1 VALUES ("2022-01-15", "Alice", "Book", 300)')
cur.execute(f'INSERT INTO Transaction1 VALUES ("2022-01-16", "Alice", "Pen", 10)')
cur.execute(f'INSERT INTO Transaction1 VALUES ("2022-01-21", "Bob", "Book", 300)')
cur.execute(f'INSERT INTO Transaction1 VALUES ("2022-01-23", "Bob", "Pen", 10)')
cur.execute(f'INSERT INTO Transaction1 VALUES ("2022-01-25", "Alice", "Pen", 10)')
conn.commit()

In [9]:
# create transaction1 table
cur.execute("CREATE TABLE Transaction2 (Date date, Customer text, Goods text, Price real)")
cur.execute(f'INSERT INTO Transaction2 VALUES ("2022-01-21", "Bob", "Cap", 1500)')
cur.execute(f'INSERT INTO Transaction2 VALUES ("2022-01-21", "Bob", "T-shirt", 1000)')
cur.execute(f'INSERT INTO Transaction2 VALUES ("2022-01-23", "Charlie", "Shoes", 2000)')
cur.execute(f'INSERT INTO Transaction2 VALUES ("2022-01-25", "Bob", "Shoes", 300)')
cur.execute(f'INSERT INTO Transaction2 VALUES ("2022-01-27", "Charlie", "T-shirt", 1000)')
conn.commit()

In [10]:
cur.execute("SELECT * FROM Customer").fetchall()

[('Alice', '2022-01-15', None, 25.0),
 ('Bob', '2022-01-20', None, 21.0),
 ('Charlie', '2022-01-21', None, 17.0)]

In [11]:
cur.execute("SELECT * FROM Transaction1").fetchall()

[('2022-01-15', 'Alice', 'Cup', 100.0),
 ('2022-01-15', 'Alice', 'Book', 300.0),
 ('2022-01-16', 'Alice', 'Pen', 10.0),
 ('2022-01-21', 'Bob', 'Book', 300.0),
 ('2022-01-23', 'Bob', 'Pen', 10.0),
 ('2022-01-25', 'Alice', 'Pen', 10.0)]

In [12]:
cur.execute("SELECT * FROM Transaction2").fetchall()

[('2022-01-21', 'Bob', 'Cap', 1500.0),
 ('2022-01-21', 'Bob', 'T-shirt', 1000.0),
 ('2022-01-23', 'Charlie', 'Shoes', 2000.0),
 ('2022-01-25', 'Bob', 'Shoes', 300.0),
 ('2022-01-27', 'Charlie', 'T-shirt', 1000.0)]

## Directory structure

Suppose that we have some jinja2 templates for sql scripts.

The structure of templates are as follows.

In [13]:
def tree(root: str, only_file: bool=False) -> Iterable[str]:
    for root_, dirs, files in os.walk(root):
        if not only_file:
            yield root_
        for file in files:
            yield os.path.join(root_, file)

In [14]:
list(tree(ROOT_DIREC))

['../sqls/',
 '../sqls/master',
 '../sqls/master\\master.sql.j2',
 '../sqls/transactions',
 '../sqls/transactions\\transaction1.sql.j2',
 '../sqls/transactions\\transaction2.sql.j2']

In [15]:
for fpath in tree(ROOT_DIREC, True):
    line: str = f"=== Content of {fpath} ==="
    print(line)
    print(load_txt(fpath))
    print("="*len(line))

=== Content of ../sqls/master\master.sql.j2 ===
SELECT 
    *
    FROM Customer
    {% if start is defined and end is defined %}
    where '{{ start }}' <= JoinDate and JoinDate <= '{{ end }}'
    {% elif start is defined %}
    where '{{ start }}' <= JoinDate
    {% elif end is defined %}
    where JoinDate <= '{{ end }}'
    {% else %}
    {% endif %}
=== Content of ../sqls/transactions\transaction1.sql.j2 ===
SELECT 
    *
    FROM Transaction1
    {% if start is defined and end is defined %}
    where '{{ start }}' <= Date and Date <= '{{ end }}'
    {% elif start is defined %}
    where '{{ start }}' <= Date
    {% elif end is defined %}
    where Date <= '{{ end }}'
    {% else %}
    {% endif %}
=== Content of ../sqls/transactions\transaction2.sql.j2 ===
SELECT 
    *
    FROM Transaction2
    {% if start is defined and end is defined %}
    where '{{ start }}' <= Date and Date <= '{{ end }}'
    {% elif start is defined %}
    where '{{ start }}' <= Date
    {% elif end is defi

## Case without DirAPI: use only jinja2

Of course, without `dirapi`, you can create an interface to load dataset from the database.

In [16]:
def load_data(
    key: str,
    env: jinja2.Environment=jinja2.Environment(loader=jinja2.FileSystemLoader(ROOT_DIREC)),
    **kwargs
) -> List[Tuple[Any, ...]]:
    
    # load template
    template = env.get_template(key)
    # render
    script = template.render(**kwargs)
    # exeucte
    with sql.connect(DB) as conn:
        cur = conn.cursor()
        return cur.execute(script).fetchall()

In [17]:
load_data("master/master.sql.j2")

[('Alice', '2022-01-15', None, 25.0),
 ('Bob', '2022-01-20', None, 21.0),
 ('Charlie', '2022-01-21', None, 17.0)]

In [18]:
load_data("master/master.sql.j2", start=date(2022, 1, 19))

[('Bob', '2022-01-20', None, 21.0), ('Charlie', '2022-01-21', None, 17.0)]

In [19]:
load_data("transactions/transaction1.sql.j2")

[('2022-01-15', 'Alice', 'Cup', 100.0),
 ('2022-01-15', 'Alice', 'Book', 300.0),
 ('2022-01-16', 'Alice', 'Pen', 10.0),
 ('2022-01-21', 'Bob', 'Book', 300.0),
 ('2022-01-23', 'Bob', 'Pen', 10.0),
 ('2022-01-25', 'Alice', 'Pen', 10.0)]

In [20]:
load_data("transactions/transaction1.sql.j2", start=date(2022,1,21), end=date(2022,1,24))

[('2022-01-21', 'Bob', 'Book', 300.0), ('2022-01-23', 'Bob', 'Pen', 10.0)]

However, in this way, you must know and remember the directory structure.
It is a little bit troublesome.

## Case with DirAPI: create an API using jinja2

With `dirapi`, you don't have to remember the directory structure.

In [21]:
def load_data_(
    path: str,
    env: jinja2.Environment=jinja2.Environment(loader=jinja2.FileSystemLoader(ROOT_DIREC)),
    **kwargs,
) -> List[Tuple[Any, ...]]:
    
    # load template
    template = env.get_template(os.path.relpath(path, ROOT_DIREC).replace(os.sep, "/"))
    # render
    script = template.render(**kwargs)
    # exeucte
    with sql.connect(DB) as conn:
        cur = conn.cursor()
        return cur.execute(script).fetchall()

In [22]:
Api = create_api(ROOT_DIREC, {"load": load_data_})

In [23]:
print(help_tree(Api))

Api
Api.Master
Api.Master.Master_sql
Api.Master.Master_sql.load
Api.Transactions
Api.Transactions.Transaction1_sql
Api.Transactions.Transaction1_sql.load
Api.Transactions.Transaction2_sql
Api.Transactions.Transaction2_sql.load



In [24]:
Api.Transactions.Transaction1_sql.load()

[('2022-01-15', 'Alice', 'Cup', 100.0),
 ('2022-01-15', 'Alice', 'Book', 300.0),
 ('2022-01-16', 'Alice', 'Pen', 10.0),
 ('2022-01-21', 'Bob', 'Book', 300.0),
 ('2022-01-23', 'Bob', 'Pen', 10.0),
 ('2022-01-25', 'Alice', 'Pen', 10.0)]

In [25]:
Api.Transactions.Transaction1_sql.load(start=date(2022,1,21), end=date(2022,1,24))

[('2022-01-21', 'Bob', 'Book', 300.0), ('2022-01-23', 'Bob', 'Pen', 10.0)]

Good! We got the same result.

The API structure is obvious with `print(help_tree(Api))`.

We don't have to remember it.